In [1]:
from IPython.core.interactiveshell import InteractiveShell  #执行该代码可以使得当前nb支持多输出
InteractiveShell.ast_node_interactivity = "all" 
import numpy as np
import pandas as pd 
pd.options.display.max_rows = 8  
import requests
import parsel
from bs4 import BeautifulSoup
from lxml import etree
from time import sleep
from selenium.webdriver.chrome.service import Service
from selenium.webdriver import Chrome,ChromeOptions
from selenium.webdriver.common.by import By
import warnings
import csv
import random

#忽略警告
warnings.filterwarnings("ignore")

In [2]:
# 设置请求头
headers = {
    # 自己电脑的代理通过在 浏览器输入 chrome://version/ 可以找到
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36'
}

# 获取网页内容
url = "https://www.cnipa.gov.cn/tjxx/jianbao/year2023/a.html"

response = requests.get(url,headers=headers)
html_content = response.text





In [3]:

selector = parsel.Selector(text=html_content)
div_elements = selector.css('div')

lk = []
for div in div_elements:
    link = div.css('a::attr(href)').get()
    if link:
        link="https://www.cnipa.gov.cn/tjxx/jianbao/year2023/"+link
        lk.append(link)
link_array = np.array(lk)[3:,]

print(link_array)

['https://www.cnipa.gov.cn/tjxx/jianbao/year2023/a/a1.html'
 'https://www.cnipa.gov.cn/tjxx/jianbao/year2023/a/a2.html'
 'https://www.cnipa.gov.cn/tjxx/jianbao/year2023/a/a3.html'
 'https://www.cnipa.gov.cn/tjxx/jianbao/year2023/a/a4.html'
 'https://www.cnipa.gov.cn/tjxx/jianbao/year2023/a/a5.html'
 'https://www.cnipa.gov.cn/tjxx/jianbao/year2023/a/a6.html'
 'https://www.cnipa.gov.cn/tjxx/jianbao/year2023/a/a7.html'
 'https://www.cnipa.gov.cn/tjxx/jianbao/year2023/a/a8.html'
 'https://www.cnipa.gov.cn/tjxx/jianbao/year2023/a/a9.html'
 'https://www.cnipa.gov.cn/tjxx/jianbao/year2023/a/a10.html'
 'https://www.cnipa.gov.cn/tjxx/jianbao/year2023/a/a11.html']


In [4]:
# 发送 GET 请求获取网页内容
url = link_array[0]
response = requests.get(url,headers=headers)
# 使用 BeautifulSoup 解析 HTML 内容
soup = BeautifulSoup(response.text, 'html.parser')
tables = soup.find_all('table')
specified_tables = tables[1]
# print(specified_table.prettify())
df = pd.read_html(str(specified_tables))[0]
df=df[4:]
df.drop([4, 6], inplace=True)

df.head(6)



,0,1,2,3,4,5,6,7
5,分组 Grouping,分组 Grouping,发明 Invention,发明 Invention,实用新型 Utility Model,实用新型 Utility Model,外观设计 Industrial Design,外观设计 Industrial Design
7,分组 Grouping,分组 Grouping,数量 Number,构成 %,数量 Number,构成 %,数量 Number,构成 %
8,合计 Total,申请量Application,1677701,30.2%,3063928,55.1%,820361,14.7%
9,合计 Total,申请量Application,1677701,30.2%,3063928,55.1%,820361,14.7%
10,合计 Total,授权量 Grant,920797,25.2%,2090331,57.3%,637944,17.5%
11,合计 Total,授权量 Grant,920797,25.2%,2090331,57.3%,637944,17.5%


In [5]:
#一些数据处理操作

df.set_index(df.columns[0],inplace=True)
df.index.name = None
df.columns = df.iloc[0]
df = df.iloc[1:]
df.columns.name = None

df

,分组 Grouping,发明 Invention,发明 Invention,实用新型 Utility Model,实用新型 Utility Model,外观设计 Industrial Design,外观设计 Industrial Design
分组 Grouping,分组 Grouping,数量 Number,构成 %,数量 Number,构成 %,数量 Number,构成 %
合计 Total,申请量Application,1677701,30.2%,3063928,55.1%,820361,14.7%
合计 Total,申请量Application,1677701,30.2%,3063928,55.1%,820361,14.7%
合计 Total,授权量 Grant,920797,25.2%,2090331,57.3%,637944,17.5%
...,...,...,...,...,...,...,...
国外 Foreign,授权量 Grant,101563,87.0%,5667,4.9%,9560,8.2%
国外 Foreign,有效量 In Force,901938,84.0%,52920,4.9%,118308,11.0%
国外 Foreign,有效量 In Force,901938,84.0%,52920,4.9%,118308,11.0%
"注：有效量为报告期末处于专利权维持状态的案卷数量。统计范围为：发明、实用新型、外观设计。与申请量和授权量不同，有效量是存量数据而非流量数据。 Note: Statistics on patents in force refer to the number of patents which are still in force at the time of data collection, which include patents for invention, utility model and industrial design. Unlike the number of patent applications or grants, the number of patents in force is not flowing data but stocking data.",注：有效量为报告期末处于专利权维持状态的案卷数量。统计范围为：发明、实用新型、外观设计。与申...,注：有效量为报告期末处于专利权维持状态的案卷数量。统计范围为：发明、实用新型、外观设计。与申...,注：有效量为报告期末处于专利权维持状态的案卷数量。统计范围为：发明、实用新型、外观设计。与申...,注：有效量为报告期末处于专利权维持状态的案卷数量。统计范围为：发明、实用新型、外观设计。与申...,注：有效量为报告期末处于专利权维持状态的案卷数量。统计范围为：发明、实用新型、外观设计。与申...,注：有效量为报告期末处于专利权维持状态的案卷数量。统计范围为：发明、实用新型、外观设计。与申...,注：有效量为报告期末处于专利权维持状态的案卷数量。统计范围为：发明、实用新型、外观设计。与申...


In [6]:
# 将原有索引设置为一级索引，同时将第一列和第一行作为二级索引
df = df.set_index(df.columns[0], append=True)
df.index.name = None  # 删除索引列的名称
df.columns = pd.MultiIndex.from_arrays([df.iloc[0], df.columns], names=[None, None])  # 创建二级索引
df = df.iloc[1:]  # 删除原始的第一行
df = df.swaplevel(axis=1)
df

发明 Invention  \
                                                                                                                                               数量 Number   
                                                   分组 Grouping                                                                                             
合计 Total                                           申请量Application                                                                                1677701   
                                                   申请量Application                                                                                1677701   
                                                   授权量 Grant                                                                                      920797   
                                                   授权量 Grant                                                                                      920797   
...                                                                                                                                                  ...   
国外 Foreign                                         授权量 Grant                                                                                      101563   
                                                   有效量 In Force                                                                                   901938   
                                                   有效量 In Force                                                                                   901938   
注：有效量为报告期末处于专利权维持状态的案卷数量。统计范围为：发明、实用新型、外观设计。与申请... 注：有效量为报告期末处于专利权维持状态的案卷数量。统计范围为：发明、实用新型、外观设计。与申请...  注：有效量为报告期末处于专利权维持状态的案卷数量。统计范围为：发明、实用新型、外观设计。与申...   

                                                                                                                                                          \
                                                                                                                                                    构成 %   
                                                   分组 Grouping                                                                                             
合计 Total                                           申请量Application                                                                                  30.2%   
                                                   申请量Application                                                                                  30.2%   
                                                   授权量 Grant                                                                                       25.2%   
                                                   授权量 Grant                                                                                       25.2%   
...                                                                                                                                                  ...   
国外 Foreign                                         授权量 Grant                                                                                       87.0%   
                                                   有效量 In Force                                                                                    84.0%   
                                                   有效量 In Force                                                                                    84.0%   
注：有效量为报告期末处于专利权维持状态的案卷数量。统计范围为：发明、实用新型、外观设计。与申请... 注：有效量为报告期末处于专利权维持状态的案卷数量。统计范围为：发明、实用新型、外观设计。与申请...  注：有效量为报告期末处于专利权维持状态的案卷数量。统计范围为：发明、实用新型、外观设计。与申...   

                                                                                                                                      实用新型 Utility Model  \
                                                                                                                                               数量 Number   
                                                   分组 Grouping                    

In [9]:
df.to_excel("./实验一爬取数据/国内外三种专利申请受理_授权_有效累计状况.xlsx",header=True,index=True)

In [10]:
#####  登录微博
# 导入需要的库
from selenium import webdriver  # Selenium用于模拟浏览器操作
from selenium.webdriver.common.keys import Keys  
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains  
import time  

In [53]:
# 启动浏览器
browser = webdriver.Chrome()  # 启动Chrome浏览器，确保已经安装Chrome浏览器和对应的ChromeDriver驱动程序

# 打开微博首页
browser.get("https://passport.weibo.com/sso/signin?entry=miniblog&source=miniblog&disp=popup&url=https%3A%2F%2Fweibo.com%2Fnewlogin%3Ftabtype%3Dweibo%26gid%3D102803%26openLoginLayer%3D0%26url%3Dhttps%253A%252F%252Fweibo.com%252F")  # 使用浏览器打开微博首页

# 等待页面加载完成
time.sleep(3)  

# 定位要点击的元素
button = browser.find_element(By.XPATH, "//*[@id='app']/div/div/div[2]/div[2]/ul/li[2]/a/span[2]")  # 用XPath定位元素，可以根据实际情况修改 //*[@id="app"]/div/div/div[2]/div/ul/li[2]/a/span[1]
# 执行点击操作
button.click()

# 定位账号输入框，并填入账号
user_input = browser.find_element(By.XPATH,"//*[@id='app']/div/div/div[2]/div[2]/form/div[1]/input")  # 使用id属性定位账号输入框
user_input.send_keys("17763769712")  # 向账号输入框输入你的用户名

# 定位密码输入框，并填入密码
pwd_input = browser.find_element(By.XPATH,"//*[@id='app']/div/div/div[2]/div[2]/form/div[2]/input")  # 使用name属性定位密码输入框
pwd_input.send_keys("Yyzz20030618")  # 向密码输入框输入你的密码

# 等待登录完成
time.sleep(3)  # 等待10秒钟，确保登录成功和页面跳转完成，这里的时间可以根据实际情况进行调整

# 定位要点击的是登录按钮
button = browser.find_element(By.XPATH,"//*[@id='app']/div/div/div[2]/div[2]/button")
# 点击登录按钮
button.click()

# # 滚动条拉到指定位置（具体元素）  
# target = browser.find_element(By.XPATH,"//*[@class='slider']")  
# browser.execute_script("arguments[0].scrollIntoView();", target)  

time.sleep(3)  
cookies = browser.get_cookies()
print(cookies)
# browser.close() 

[{'domain': '.passport.weibo.com', 'httpOnly': False, 'name': 'X-CSRF-TOKEN', 'path': '/', 'sameSite': 'Lax', 'secure': True, 'value': 'CK-Yco0wcag-C_e9J_FRx2u9pW0-Dac2CWwzE7CzN_4mgoeYN7e='}]


In [50]:
######北京知识产权官方微博数据抓取
import requests
import csv
from lxml import etree
import time
import random
from datetime import datetime
import os
import pandas as pd

In [51]:
def extract_full_content(page):
    headers = {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36 Edg/114.0.1823.51',
        'cookie': '_T_WM=29007828263; SUBP=0033WrSXqPxfM725Ws9jqgMF55529P9D9WFPrQQDSM8GU-LDwXGJfO-75JpX5K-hUgL.FoMRSh-f1hqNSoq2dJLoI7p0dNHjdg8X9HvodGnt; SSOLoginState=1687401343; ALF=1689993343; SCF=AliNZPJfd8jrC1kjeXX5Wln4GfwuiZk5G2brx6c1VvQ_NS49PAH7r8V1CrDXL_YlLEmhT_4y5HWR4ywbGEllBaM.; SUB=_2A25Jl8MvDeRhGeFG71cU-CjLzTqIHXVre-1nrDV6PUJbktANLXT_kW1NeWK98E_aztGw8jkuazsPlMRFEQEXXDZy'
    }
    content = page.xpath('./div/span[@class="ctt"]//text()')

    full_content = ''
    for i in range(len(content)):
        full_content += content[i]

    if u'全文' in full_content:
        full_content_link = page.xpath('./div/span[@class="ctt"]/a/@href')
        content_url = 'https://weibo.cn' + full_content_link[-1]
        response = requests.get(url=content_url, headers=headers)
        tree = etree.HTML(response.text.encode('utf-8'))
        div = tree.xpath('//*[@id="M_"]/div[1]/span[@class="ctt"]//text()')
        full_content = ''
        for j in range(len(div)):
            full_content += div[j]
        return full_content
    else:
        return full_content


def format_time(time_info):
    now = datetime.now()
    if '今天' in time_info:
        time_str = time_info.replace('今天', now.strftime('%Y-%m-%d'))
    elif '月' in time_info and '日' in time_info:
        month = time_info.split('月')[0]
        day = time_info.split('月')[1].split('日')[0]
        time_str = now.strftime('%Y') + '-' + month.zfill(2) + '-' + day.zfill(2)
    else:
        time_str = time_info
    return time_str


def extract_info(page):
    content = extract_full_content(page)
    time_info = page.xpath('.//span[@class="ct"]/text()')[0].split('\xa0')[0]
    time_str = format_time(time_info)
    source = page.xpath('.//span[@class="ct"]/text()')[0].split('\xa0')[1][2:]
    like_count = page.xpath('.//a[contains(text(), "赞[")]/text()')[0][2:-1]
    comment_count = page.xpath('.//a[contains(text(), "评论[")]/text()')[0][3:-1]
    repost_count = page.xpath('.//a[contains(text(), "转发[")]/text()')[0][3:-1]
    return content, time_str, source, like_count, comment_count, repost_count

In [52]:
url = 'https://weibo.cn/2611704875?mp=1302&page='
base_url = 'https://weibo.cn/2611704875?mp=1302&page='

headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36 Edg/125.0.0.0',
    'cookie': '_T_WM=8f1c0a26d8ada0f4491205a111d59473; SCF=Alzst0ONGEqytA_Ri0CzQj0KKN9QRFV9lcOiz2ZJ7xKiYrZnDRpnPqWdtgbyG8KAn0wfqZ9VtQPZk8EAnEHRy7I.; SUB=_2A25LSxo6DeRhGeFH6lsZ9yvIyD-IHXVoKRPyrDV6PUJbktANLWjwkW1Ne9dAHCZXYumFfl_qjyxmUhO7cAm8qCoA; SUBP=0033WrSXqPxfM725Ws9jqgMF55529P9D9WWJLVu-LI4uxDCyKD4XGJqV5JpX5KMhUgL.FoM4eK.RS0-Xe0e2dJLoIp7LxKML1KBLBKnLxKqL1hnLBoMfeo2NSh.ESh-X; SSOLoginState=1716480618; ALF=1719072618'
}

filename = '北京知识产权微博内容.csv'

# Delete the existing file if it exists
if os.path.exists(filename):
    os.remove(filename)
    print('Existing data.csv file deleted.')

with open(filename, mode='a', encoding='utf-8', newline='') as file:
    csv_writer = csv.writer(file)
    page_number = 1
    while True:
        time.sleep(random.randint(1, 8))
        current_url = base_url + str(page_number)+"&rand="+str(random.randint(1, 10000))+"&p=r"
        try:
            response = requests.get(url=current_url, headers=headers)
        except ConnectionError:
            print('连接失败，跳过该页:', current_url)
            continue
        tree = etree.HTML(response.text.encode('utf-8'))
        data_list = tree.xpath('/html/body/div[@class="c"]')
        if not data_list:
            break  # No more pages, exit the loop
        del data_list[-1]
        print('正在爬取第{}页'.format(page_number))
        for data in data_list:
            content, time_info, source, like_count, comment_count, repost_count = extract_info(data)
            csv_writer.writerow([content, time_info, source, like_count, comment_count, repost_count])
        page_number += 1

print('Extraction completed.')

Extraction completed.


In [21]:
df = pd.read_csv('北京知识产权微博内容.csv')
# 创建新的列名列表
new_columns = ['微博内容', '时间', '来源', '点赞数', '评论数', '转发数']

# 将列名作为内容添加到新的行
df.loc[-1] = df.columns
df.index = df.index + 1
df.sort_index(inplace=True)

# 重命名列名
df.columns = new_columns
df.head(20)

,微博内容,时间,来源,点赞数,评论数,转发数
0,【重磅！关系保交房、个人房贷，最新政策要点汇总→】国务院新闻办公室于2024年5月17日下午...,2024-05-20,微博网页版,0,0.1,0.2
1,【习近平同俄罗斯总统普京共同会见记者】5月16日，国家主席习近平在北京人民大会堂同俄罗斯总统...,2024-05-16,iPhone客户端,0,0,0
2,【习近平同匈牙利总理欧尔班举行会谈】当地时间5月9日下午，国家主席习近平在布达佩斯总理府同匈...,2024-05-11,微博网页版,1,0,0
3,【习近平抵达塞尔维亚贝尔格莱德尼古拉·特斯拉国际机场发表书面讲话】 网页链接,2024-05-08,iPhone客户端,1,0,0
4,【习近平同法国总统马克龙共同会见记者】当地时间5月6日下午，国家主席习近平在巴黎爱丽舍宫同法...,2024-05-08,iPhone客户端,0,0,0
5,NaN,2024-05-06,iPhone客户端,1,0,0
6,NaN,2024-05-06,微博网页版,0,0,0
7,今天劳动节，向每一位劳动者致敬 中国政府网的微博视频,2024-05-01,iPhone客户端,2,论[34,0
8,NaN,2024-04-30,微博网页版,1,0,0
9,NaN,2024-04-29,微博网页版,0,0,0


In [22]:
df.to_csv('北京知识产权微博内容.csv')

In [13]:
from IPython.core.interactiveshell import InteractiveShell  #执行该代码可以使得当前nb支持多输出
InteractiveShell.ast_node_interactivity = "all" 
import requests
import pandas as pd
import random
from time import sleep

In [49]:
#这个爬取没问题，但是会有连接时间问题，建议选时间跨度不要太大
province_text = '广东、河北、山西、辽宁、吉林、黑龙江、江苏、浙江、安徽、福建、江西、山东、河南、湖北、湖南、海南、四川、贵州、云南、陕西、甘肃、青海、台湾、内蒙古、广西、西藏、宁夏、新疆、北京、天津、上海、重庆、香港、澳门'
all_province_list = list(province_text.split('、'))
df = pd.DataFrame(dtype=str)

for province in all_province_list:
    for i in range(2020,2025):
        url = 'https://www.baiten.cn/results/filter'
        params = {
            'q': 'aa:((' + province + ')) AND (pd:[' + str(i) + '0101 TO ' + str(i) + '1231])',
            'fq': '',
            'sc': '35184372088831'
        }
        headers = {
            'Cookie': '_uab_collina=171636084297048745836638; yunsuo_session_verify=651468e84a027b9680fb898cb3261dac; JSESSIONID=467E8A8E43574AD6ADDD2CA1AA8E598B; zlcp2024423=true; BSESSION=e2148644ecde040ae13f44b2ace4184a38f0d6b0623961ad; PD=a6e2323d0e6adfd16b0bb9a0e28457c5d0c7bf66bf1f72327d2d56e14cb82af5a5f1e3e7ae91cf98c7fa13fa30f3d1fd'
        }
        resp = requests.post(url=url,params=params,headers=headers)
        page_json = resp.json()
        resp.close()
        
        cn_um,cn_id,cn_in,cn_gp,cn_in_gp = 0,0,0,0,0
        try:    
            cn_um = int(page_json['facetPivots']['country']['cn']['cn_um'])
        except:
            print(province,i,'实用新型专利')
        try:
            cn_id = int(page_json['facetPivots']['country']['cn']['cn_id'])
        except:
            print(province,i,'外观专利')
        try:
            cn_in = int(page_json['facetPivots']['country']['cn']['cn_in'])
        except:
            print(province,i,'发明专利')
        try:
            cn_gp = int(page_json['facetPivots']['country']['cn']['cn_gp'])
        except:
            print(province,i,'发明授权专利')
        try:
            cn_in_gp = int(page_json['facetPivots']['country']['cn']['cn_in_gp'])
        except:
            print(province,i,'发明公开专利')
        
        df = pd.concat([df,pd.DataFrame({'发明专利':cn_in,'发明公开专利':cn_in_gp,'发明授权专利':cn_gp,
                                         '外观专利':cn_id,'实用新型专利':cn_um,'年份':i,'省份':province},
                                       index=[len(df)])],axis=0)
    sleep(random.randint(1,3))
    
df
df.to_excel('./实验一爬取数据/佰腾网/佰腾网2020-2025全国专利数据.xlsx')
            


,发明专利,发明公开专利,发明授权专利,外观专利,实用新型专利,年份,省份
0,218942,114834,104108,258202,381193,2020,广东
1,261279,137529,123750,285165,484819,2021,广东
2,246477,148772,97705,264559,458065,2022,广东
3,254422,201878,52544,235930,325327,2023,广东
...,...,...,...,...,...,...,...
166,168,79,89,63,187,2021,澳门
167,173,89,84,44,232,2022,澳门
168,213,171,42,40,171,2023,澳门
169,322,291,31,17,143,2024,澳门


In [15]:
import os
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [43]:
start_date = "20250101"
end_date = "20250102"

# 创建一个空的DataFrame用于存储数据
df = pd.DataFrame(columns=['网址', '标题', '时间'])

# 设置请求头(headers)
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36 Edg/114.0.1823.51',
    'Cookie': 'Hm_lvt_162a2ae8b540393bd5792dda2692396e=1687362784; PHPSESSID=214cd4ed0774bc1bd19001d1640ebe75; uuid=CgI1QGSTHOEAMh3oECWbAg==; Hm_lpvt_162a2ae8b540393bd5792dda2692396e=1687447774'
}

# 循环遍历日期范围
current_date = start_date

while current_date <= end_date:
    date = pd.to_datetime(current_date, format="%Y%m%d").strftime("%Y-%m-%d")
    print('\n')
    print("===================" + date + "===================")
    url = f"https://bjrbdzb.bjd.com.cn/bjrb/mobile/2025/{current_date}/{current_date}_m.html#page0"
    response = requests.get(url, headers=headers, cookies=headers)
    soup = BeautifulSoup(response.content, 'html.parser')

    # 提取网址和标题
    links = soup.find_all('a', attrs={'data-href': True})
    for index, link in enumerate(links, start=1):
        mid_l = len(df.index)
        href = link['data-href']
        href = "https://bjrbdzb.bjd.com.cn/bjrb/mobile/2025/" + current_date + "/" + href[2:]  # 更新网址格式
        title = link.text.strip()

        # 将数据添加到DataFrame中
#         df = df.append({'网址': href, '标题': title, '时间': current_date}, ignore_index=True)
        df.loc[mid_l] = [href,title,current_date]


        response = requests.get(href, headers=headers, cookies=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        content_b = soup.find_all('b')
        b_texts = [tag.get_text() for tag in content_b]
        content_p = soup.find_all('p')
        p_texts = [tag.get_text() for tag in content_p]


        with open("./实验一爬取数据/北京日报/"+b_texts[0]+".txt",'w') as f:
            f.write(b_texts[0]+"\n")
            for i in p_texts:
                f.write(i+"\n")
        
        

        # 打印保存进度
        print(f"已将{date}第{index}篇文章中{title}写入北京日报文件夹")

    # 更新日期
    current_date = pd.to_datetime(current_date, format="%Y%m%d")
    current_date += pd.DateOffset(days=1)
    current_date = current_date.strftime("%Y%m%d")

# 将DataFrame保存到Excel文件
df.to_excel("./实验一爬取数据/Python实验一省机关报数据的抓取（以北京日报为例）.xlsx", index=False)



===================2025-01-01===================


23

1

50

9

...

200

92

11

14

已将2025-01-02第49篇文章中小院儿“班主任”写入北京日报文件夹


In [22]:
print(links)

[<a data-href="./20250201_001/content_20250201_001_1.htm#page0" data-newid="bjrb2025020100101">假期前四天接待游客853.6万人次</a>, <a data-href="./20250201_001/content_20250201_001_2.htm#page0" data-newid="bjrb2025020100102">注重家庭，注重家教，注重家风</a>, <a data-href="./20250201_001/content_20250201_001_3.htm#page0" data-newid="bjrb2025020100103">消费供给丰富 节日消费升温</a>, <a data-href="./20250201_001/content_20250201_001_4.htm#page0" data-newid="bjrb2025020100104">一起过年亲上加亲</a>, <a data-href="./20250201_001/content_20250201_001_5.htm#page0" data-newid="bjrb2025020100105">唐山皮影剧团蛇年开箱来京演绎《大闹天宫》</a>, <a data-href="./20250201_001/content_20250201_001_6.htm#page0" data-newid="bjrb2025020100106">天坛韶乐连古今</a>, <a data-href="./20250201_002/content_20250201_002_1.htm#page1" data-newid="bjrb2025020100201">为欢乐时光筑起一道“防火墙”</a>, <a data-href="./20250201_002/content_20250201_002_2.htm#page1" data-newid="bjrb2025020100202">“春语”暖了外籍旅客</a>, <a data-href="./20250201_002/content_20250201_002_3.htm#page1" data-newid="bjrb2025020100203">古镇

16

86

18

144

250

...

222

173

249

23

260